## 搜索模式和基础知识
使用 Wea​​viate，您可以使用[向量相似性搜索](https://weaviate.io/developers/weaviate/search/similarity)、[关键字搜索](https://weaviate.io/developers/weaviate/search/bm25)或[两者结合的混合搜索](https://weaviate.io/developers/weaviate/search/hybrid)来查询数据。您可以控制要返回的对象属性和元数据。

In [1]:
import json
import weaviate
from weaviate.auth import AuthApiKey

# 连接到本地部署的 Weaviate
client = weaviate.Client(
    url="http://127.0.0.1:8080",
    auth_client_secret=AuthApiKey("WVF5YThaHlkYwhGUSmCRgsX3tD5ngdN8pkih")
)

## 列出对象

In [2]:
### V3
response = (
    client.query.get("JeopardyQuestion", ["question"])
    .with_limit(1)
    .do()
)

print(response)

{'data': {'Get': {'JeopardyQuestion': [{'question': 'This vector DB is OSS and supports automatic property type inference on import'}]}}}


In [ ]:
####V4
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=1
)

for o in response.objects:
    print(o.properties)

## limit返回对象
用于limit设置要返回的固定最大对象数。

In [3]:
### V3
response = (
    client.query.get("JeopardyQuestion", ["question"])
    .with_limit(1)
    .do()
)

print(response)

{'data': {'Get': {'JeopardyQuestion': [{'question': 'This vector DB is OSS and supports automatic property type inference on import'}]}}}


In [ ]:
##V4
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=1
)

for o in response.objects:
    print(o.properties)

## 指定对象properties
您可以指定要返回的对象属性。

In [5]:
## V3
response = (
    client.query
    .get("JeopardyQuestion", ["question", "answer"])
    .with_limit(1).do()
)

print(response)

{'data': {'Get': {'JeopardyQuestion': [{'answer': 'Weaviate', 'question': 'This vector DB is OSS and supports automatic property type inference on import'}]}}}


In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=1,
    return_properties=["question", "answer", "points"]
)

for o in response.objects:
    print(o.properties)

## 检索对象vector
您可以检索对象向量。（也适用于使用命名向量的情况。）

In [6]:
## V3
response = (
    client.query.get("Test")
    .with_additional("vector")
    .with_limit(1).do()
)

print(response)

{'data': {'Get': {'Test': [{'_additional': {'vector': [0.015235687, 0.0016991721, 0.039902743, -0.039280858, 0.014814094, -0.01826017, -0.009091843, 0.0036597254, -0.0028348505, 0.00871796, -0.012907752, -0.0055012847, 0.023301642, -0.009656847, -0.03313284, 0.025604602, -0.01784021, -0.008317092, 0.0072799423, -0.0063043116, 0.03260691, -0.0505275, -0.01166349, -0.024207281, 0.0107690925, 0.022621175, -0.014303998, -0.0035883393, 0.038469717, -0.032654095, -0.045827378, -0.0024251773, -0.022272935, 0.07812713, 0.052516785, -0.029790198, -0.015588896, 0.03173043, -0.028983798, -0.02220564, -0.013970379, 0.0026310172, -0.037356574, -0.024574248, 0.0032432035, 0.014139596, -0.015757026, -0.017132943, 0.040912487, 0.016084787, 0.029948648, 0.042931568, 0.028160024, -0.019590521, 0.015302704, -0.0028663094, -0.03002561, -0.017931553, -0.042967808, -0.012673756, 0.01723069, 0.07118438, 0.033718746, -0.01826973, -0.016310565, -0.016551884, 0.03273845, -0.007812507, 0.034855675, 0.018934432, 

In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    include_vector=True,
    limit=1
)

print(response.objects[0].vector["default"])

## 检索对象id
您可以检索对象id（uuid）。

In [7]:
response = (
    client.query.get("JeopardyQuestion")
    .with_additional("id")
    .with_limit(1).do()
)

print(response)

{'data': {'Get': {'JeopardyQuestion': [{'_additional': {'id': '12345678-e64f-5d94-90db-c8cfa3fc1234'}}]}}}


In [ ]:
### V4
jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    # Object IDs are included by default with the `v4` client! :)
    limit=1
)

for o in response.objects:
    print(o.uuid)

## 检索交叉引用的属性
要从交叉引用的对象中检索属性，请指定：

- 交叉引用属性
- 目标交叉引用集合
- 要检索的属性

In [8]:
## V3
response = (
    client.query
    .get(
        "JeopardyQuestion",
        ["question", "hasCategory { ... on JeopardyCategory { title } }"],
    )
    .with_limit(2).do()
)

print(json.dumps(response, indent=2))

{
  "errors": [
    {
      "locations": [
        {
          "column": 43,
          "line": 1
        }
      ],
      "message": "Cannot query field \"hasCategory\" on type \"JeopardyQuestion\".",
      "path": null
    },
    {
      "locations": [
        {
          "column": 64,
          "line": 1
        }
      ],
      "message": "Unknown type \"JeopardyCategory\". Did you mean \"JeopardyQuestion\"?",
      "path": null
    }
  ]
}


In [ ]:
### V4
from weaviate.classes.query import QueryReference

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    return_references=[
        QueryReference(
            link_on="hasCategory",
            return_properties=["title"]
        ),
    ],
    limit=2
)

for o in response.objects:
    print(o.properties["question"])
    # print referenced objects
    for ref_obj in o.references["hasCategory"].objects:
        print(ref_obj.properties)

## 检索元数据值
您可以指定要返回的元数据字段。

In [9]:
## V3
response = (
    client.query.get("JeopardyQuestion", ["question"])
    .with_limit(1)
    .with_additional("creationTimeUnix")
    .do()
)

print(json.dumps(response, indent=2))

{
  "data": {
    "Get": {
      "JeopardyQuestion": [
        {
          "_additional": {
            "creationTimeUnix": "1744127869833"
          },
          "question": "This vector DB is OSS and supports automatic property type inference on import"
        }
      ]
    }
  }
}


In [ ]:
## V4
from weaviate.classes.query import MetadataQuery

jeopardy = client.collections.get("JeopardyQuestion")
response = jeopardy.query.fetch_objects(
    limit=1,
    return_metadata=MetadataQuery(creation_time=True)
)

for o in response.objects:
    print(o.properties)  # View the returned properties
    print(o.metadata.creation_time)  # View the returned creation time

## 多租户
如果启用了多租户，请在每个查询中指定租户参数。

In [ ]:
### V3
results = (
    client.query.get("MultiTenancyClass", ["property1", "property2"]).with_limit(1)
    .with_tenant("tenantA")
    .do()
)

In [ ]:
### V4
# Connect to the collection
mt_collection = client.collections.get("WineReviewMT")

# Get the specific tenant's version of the collection
collection_tenant_a = mt_collection.with_tenant("tenantA")

# Query tenantA's version
response = collection_tenant_a.query.fetch_objects(
    return_properties=["review_body", "title"],
    limit=1,
)

print(response.objects[0].properties)

## 复制
对于启用了复制的集合，您可以在查询中指定一致性级别。这适用于 CRUD 查询和搜索。

In [ ]:
import weaviate
from weaviate.data.replication import ConsistencyLevel

client = weaviate.Client("http://localhost:8080")

data_object = (
  client.data_object.get_by_id(
    uuid="36ddd591-2dee-4e7e-a3cc-eb86d30a4303",
    class_name="MyClass",
    consistency_level=ConsistencyLevel.ONE,
  )
)

# The parameter "consistency_level" can be one of ConsistencyLevel.ALL,
# ConsistencyLevel.QUORUM (default), or ConsistencyLevel.ONE. Determines how many
# replicas must acknowledge a request before it is considered successful.

print(data_object)

In [ ]:
### V4
from weaviate.classes.config import ConsistencyLevel

questions = client.collections.get(collection_name).with_consistency_level(
    consistency_level=ConsistencyLevel.QUORUM
)
response = collection.query.fetch_object_by_id("36ddd591-2dee-4e7e-a3cc-eb86d30a4303")

# The parameter passed to `withConsistencyLevel` can be one of:
# * 'ALL',
# * 'QUORUM' (default), or
# * 'ONE'.
#
# It determines how many replicas must acknowledge a request
# before it is considered successful.

for o in response.objects:
    print(o.properties)  # Inspect returned objects